# Лабораторная работа №5: Машины опорных векторов  

*Модели*: PCR, Случайный лес, kNN

*Данные*: (источник: https://raw.githubusercontent.com/ania607/ML/main/data/default_of_credit_card_clients.csv)

# Загружаем пакеты

In [1]:
# загрузка пакетов: инструменты -----------------------
#  работа с массивами
import numpy as np
#  фреймы данных
import pandas as pd
#  графики
import matplotlib as mpl
#  стили и шаблоны графиков на основе matplotlib
import seaborn as sns

# загрузка пакетов: модели ----------------------------
# метод частных наименьших квадратов
from sklearn.cross_decomposition import PLSRegression
#  логистическая регрессия (ММП)
from sklearn.linear_model import LogisticRegression
#  линейный дискриминантный анализ (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#  квадратичный дискриминантный анализ (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#  матрица неточностей
from sklearn.metrics import classification_report, confusion_matrix
#  визуализация матрицы неточностей
from sklearn.metrics import plot_confusion_matrix
#  PPV (TP / (TP + FP))
from sklearn.metrics import precision_score
#  расчёт TPR, SPC, F1
from sklearn.metrics import precision_recall_fscore_support
#  ROC-кривая
from sklearn.metrics import plot_roc_curve, roc_curve, auc, RocCurveDisplay
#  подготовка матрицы X для модели регрессии
from statsmodels.api import add_constant
#  модель логистической регрессии
from statsmodels.formula.api import logit
#  стандартизация показателей
from sklearn.preprocessing import StandardScaler
#  метод главных компонент
from sklearn.decomposition import PCA
#  для таймера
import time
# для перекрёстной проверки и сеточного поиска
from sklearn.model_selection import KFold, GridSearchCV
# случайный лес
from sklearn.ensemble import RandomForestClassifier
#  дерево классификации
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
# бэггинг
from sklearn.ensemble import BaggingClassifier
# kNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
# конвейеры
from sklearn.pipeline import make_pipeline
# перекрёстная проверка и метод проверочной выборки 
from sklearn.model_selection import cross_val_score


In [2]:
# константы
#  ядро для генератора случайных чисел
my_seed = 13

In [3]:
# загружаем таблицу и превращаем её во фрейм
fileURL = 'https://raw.githubusercontent.com/ania607/ML/main/data/default_of_credit_card_clients.csv'
DF_all = pd.read_csv(fileURL)

# выясняем размерность фрейма
print('Число строк и столбцов в наборе данных:\n', DF_all.shape)

# наблюдения для моделирования
DF = DF_all.sample(frac=0.85, random_state=my_seed)
# отложенные наблюдения
DF_predict = DF_all.drop(DF.index)

Число строк и столбцов в наборе данных:
 (30000, 24)


In [4]:
# считаем пропуски в каждом столбце
DF_all.isna().sum()

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
Y            0
dtype: int64

Пропусков не обнаружено

In [5]:
DF.describe()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
count,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000,...,25500.000000,25500.000000,25500.000000,25500.000000,2.550000e+04,25500.000000,25500.000000,25500.000000,25500.000000,25500.000000
mean,167848.614902,1.604510,1.854078,1.551137,35.473647,-0.019294,-0.136745,-0.171020,-0.226353,-0.272706,...,43011.959020,40047.777804,38657.944118,5721.081059,5.912399e+03,5195.790706,4851.610588,4847.703137,5248.818863,0.219216
std,129894.926164,0.488965,0.791689,0.521709,9.224644,1.125633,1.197933,1.193489,1.167079,1.131698,...,63681.324672,60026.412733,59003.614301,16819.669309,2.259416e+04,16745.180224,15980.476266,15559.835697,17901.580110,0.413723
min,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2280.000000,1740.750000,1242.000000,1000.000000,8.360000e+02,390.000000,298.750000,243.750000,108.750000,0.000000
50%,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,19010.000000,18067.000000,17001.500000,2111.500000,2.010000e+03,1826.500000,1500.000000,1500.000000,1500.000000,0.000000
75%,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,54429.500000,50065.250000,49162.750000,5015.000000,5.000000e+03,4512.750000,4027.000000,4064.250000,4000.000000,0.000000
max,800000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,706864.000000,587067.000000,699944.000000,873552.000000,1.684259e+06,889043.000000,621000.000000,426529.000000,528666.000000,1.000000


# Преобразование исходных данных и построение моделей
## Стандартизация и переход к главным компонентам

Предварительно преобразуем пространство исходных показателей с помощью метода главных компонент

In [6]:
# стандартизация
sc = StandardScaler()
X_train_std = sc.fit_transform(DF.iloc[:, :24].values)

# проверяем средние и стандартные отклонения после стандартизации
for i_col in range(X_train_std.shape[1]) :
    print('Столбец ', i_col, ': среднее = ',
          np.round(np.mean(X_train_std[:, i_col]), 2),
         '   Станд. отклонение = ', 
          np.round(np.std(X_train_std[:, i_col]), 2), sep='')

Столбец 0: среднее = -0.0   Станд. отклонение = 1.0
Столбец 1: среднее = 0.0   Станд. отклонение = 1.0
Столбец 2: среднее = -0.0   Станд. отклонение = 1.0
Столбец 3: среднее = -0.0   Станд. отклонение = 1.0
Столбец 4: среднее = -0.0   Станд. отклонение = 1.0
Столбец 5: среднее = -0.0   Станд. отклонение = 1.0
Столбец 6: среднее = 0.0   Станд. отклонение = 1.0
Столбец 7: среднее = -0.0   Станд. отклонение = 1.0
Столбец 8: среднее = 0.0   Станд. отклонение = 1.0
Столбец 9: среднее = -0.0   Станд. отклонение = 1.0
Столбец 10: среднее = 0.0   Станд. отклонение = 1.0
Столбец 11: среднее = 0.0   Станд. отклонение = 1.0
Столбец 12: среднее = -0.0   Станд. отклонение = 1.0
Столбец 13: среднее = 0.0   Станд. отклонение = 1.0
Столбец 14: среднее = 0.0   Станд. отклонение = 1.0
Столбец 15: среднее = -0.0   Станд. отклонение = 1.0
Столбец 16: среднее = 0.0   Станд. отклонение = 1.0
Столбец 17: среднее = 0.0   Станд. отклонение = 1.0
Столбец 18: среднее = 0.0   Станд. отклонение = 1.0
Столбец 19: с

## Регрессия на главные компоненты (PCR)  

Пересчитаем объясняющие показатели в главные компоненты.  

In [7]:
# функция с методом главных компонент
pca = PCA()
# пересчитываем в главные компоненты (ГК)
X_train_pca = pca.fit_transform(X_train_std)

# считаем доли объяснённой дисперсии
frac_var_expl = pca.explained_variance_ratio_
print('Доли объяснённой дисперсии по компонентам в PLS:\n',
     np.around(frac_var_expl, 3),
     '\nОбщая сумма долей:', np.around(sum(frac_var_expl), 3))

Доли объяснённой дисперсии по компонентам в PLS:
 [0.272 0.175 0.065 0.062 0.044 0.04  0.038 0.038 0.037 0.035 0.032 0.031
 0.026 0.024 0.022 0.017 0.011 0.011 0.008 0.006 0.003 0.002 0.001 0.001] 
Общая сумма долей: 1.0


Первые четыре главные компоненты объясняют более 50% разброса

In [8]:
# данные для обучения моделей
X_train = DF.iloc[:, :11] 
Y_train = DF.iloc[:, -1]

# объединяем в конвейер шкалирование, ГК с 2 компонентами и логит
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components = 4),
                        LogisticRegression(random_state = my_seed, 
                                           solver = 'lbfgs'))

# будем сохранять точность моделей в один массив
score = list()
score_models = list()

# считаем точность с перекрёстной проверкой, показатель Acc
cv = cross_val_score(estimator = pipe_lr, X = X_train, y = Y_train, 
                     cv = 5, scoring='accuracy')

# записываем точность
score.append(np.around(np.mean(cv), 3)) 
score_models.append('pca_logit')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[0], ':', score[0])

Acc с перекрёстной проверкой 
для модели pca_logit : 0.796


# Случайный лес  

У модели случайного леса два настроечных параметра: количество деревьев $B$ и количество признаков для построения отдельного дерева $m$. Настроим сеточный поиск для их подбора.  

In [9]:
X = DF.drop(['Y'], axis=1)
Y = DF['Y']

# сколько столбцов в обучающих данных (p)
X_m = X.shape[1]
# возьмём значения для m: p, p/2, sqrt(p) и log2(p)
ms = np.around([X_m, X_m / 2, np.sqrt(X_m), np.log2(X_m)]).astype(int)
ms

array([23, 12,  5,  5])

In [10]:
# разбиения для перекрёстной проверки
kfold = KFold(n_splits=10, random_state=my_seed, shuffle=True)

# настроим параметры случайного леса с помощью сеточного поиска
param_grid = {'n_estimators' : list(range(10, 51, 2)),
              'max_features' : ms}

# таймер
tic = time.perf_counter()
clf = GridSearchCV(RandomForestClassifier(DecisionTreeClassifier()),
                   param_grid, scoring='accuracy', cv=kfold)
random_forest = clf.fit(X, Y)
# таймер
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 4136.02 секунд


In [11]:
# точность лучшей модели
np.around(random_forest.best_score_, 3)

0.817

In [12]:
# записываем точность
score.append(np.around(random_forest.best_score_, 3))
score_models.append('random_forest_GS')

print('Acc с перекрёстной проверкой',
      '\nдля модели', score_models[1], ':', score[1])

Acc с перекрёстной проверкой 
для модели random_forest_GS : 0.817


# Метод kNN

Реализуем метод k-ближайших соседей с преобразованием PCA.

In [13]:
k_range = list(range(1,50))
weight_options = ["uniform", "distance"]

pipe_knn = Pipeline(steps=[('standard', StandardScaler()), ('PCA', PCA(n_components = 10)), 
                           ('knn', KNeighborsClassifier())])
param_grid = {'knn__n_neighbors': list(range(1,50))}

knn = KNeighborsClassifier()
kfold = KFold(n_splits = 10, random_state = my_seed, shuffle = True)
grid = GridSearchCV(pipe_knn, param_grid, cv=kfold, scoring='accuracy')

tic = time.perf_counter()
grid.fit(X_train,Y_train)
toc = time.perf_counter()
print(f"Сеточный поиск занял {toc - tic:0.2f} секунд", sep='')

Сеточный поиск занял 164.75 секунд


In [14]:
score.append(np.around(grid.best_score_,3))
score_models.append('sc_pca_knn')

print('Acc с перекрёстной проверкой','\nдля модели',score_models[2],':',score[2])

Acc с перекрёстной проверкой 
для модели sc_pca_knn : 0.816



# Прогноз на отложенные наблюдения по лучшей модели

Ещё раз посмотрим на точность построенных моделей. 

In [15]:
# сводка по точности моделей
pd.DataFrame({'Модель' : score_models, 'Acc' : score})

,Модель,Acc
0,pca_logit,0.796
1,random_forest_GS,0.817
2,sc_pca_knn,0.816


Все модели показывают хорошую точность по показателю $Acc$, при этом самой точной оказывается модель random_. Сделаем прогноз на отложенные наблюдения.   

In [16]:
# данные для прогноза
X_pred = DF_predict.drop(['Y'], axis=1)
# строим прогноз
Y_hat = random_forest.best_estimator_.predict(X_pred)
# характеристики точности
print(classification_report(DF_predict['Y'], Y_hat))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89      3454
           1       0.67      0.36      0.47      1046

    accuracy                           0.81      4500
   macro avg       0.75      0.66      0.68      4500
weighted avg       0.79      0.81      0.79      4500

